In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set()


In [ ]:
nordic_abbs = ['SWE','FIN','NOR','DNK','ISL']

abbs = nordic_abbs

df = pd.DataFrame()

for a in abbs:
    temp = pd.read_excel('../mortality_org.xlsx',sheet_name=a,skiprows=2)
    
    df = pd.concat([df,temp],axis=0)
    
df

In [ ]:
df = df.loc[df['Sex'] == 'b']
df = df.set_index(['Country','Year','Week'])


In [ ]:
df_all = df.copy()
df = df[['Total','Total.1']]
df

In [ ]:
2364 / 0.013855 * 52

df['pop'] = (df['Total'] / df['Total.1'] * 52).astype(int)
nordic_deaths = df.groupby(['Country','Year'])['Total'].sum()
nordic_deaths

In [ ]:
nordic_pop = df.groupby(['Country','Year'])['pop'].apply(lambda x : x[0])
nordic_pop

In [ ]:
nordic_mortality = (nordic_deaths / nordic_pop).unstack().T.dropna()
nordic_mortality

In [ ]:
nordic_mortality_baseline = nordic_mortality.loc['2015' : '2018'].mean()
nordic_mortality_baseline

In [ ]:
title = 'Mortality Nordic Countries 2007 - 2020\nDataSource : mortality.org'
nordic_mortality.plot(figsize=(18,12),style='o--',title=title)
plt.ylabel('mortality')
plt.savefig('mortality_org_nordics_2007_2020.jpg',format='jpg')

In [ ]:
title = 'Mortality Nordic Countries 2007-2020 vs baseline 2015-2018\nDataSource : mortality.org'
axes = nordic_mortality.plot(figsize=(18,12),style='ro--',title=title,subplots=True,sharey=True)

for i,a in enumerate(axes):
    a.axhline(nordic_mortality_baseline.iloc[i],color='k',ls='dashed')
    a.set_ylabel('mortality')
    
plt.savefig('mortality_org_nordics_trend_vs_baseline.jpg',format='jpg')

In [ ]:
growth_vs_baseline = nordic_mortality / nordic_mortality_baseline
growth_vs_baseline

In [ ]:
title='Nordic Countries : Mortality Growth vs baseline 2015-2018\nDataSource : mortality.org'
growth_vs_baseline.plot(figsize=(18,12),style='o--',title=title)
plt.ylabel('growth factor')
plt.yticks(np.arange(0.9,1.2,0.01))
plt.axhline(1,ls='dashed',color='k')
plt.savefig('mortality_org_nordics_growth.jpg',format='jpg')

In [ ]:
nordic_mortality.loc[2020].plot(kind='bar',figsize=(18,12))
_= plt.yticks(np.arange(0,0.012,0.0005))

In [ ]:
nordic_mortality

In [ ]:
df_all

In [ ]:
### PATTERN ###
### group the 2 sets of age group cols, one with deaths, one with mortality, to combined age groups ###
def combine_ages(col):
    if '0-14' in col:
        return '0-14'
    
    if '15-64' in col:
        return '15-64'
    
    if '65-74' in col:
        return '65-74'
    
    if '75-84' in col:
        return '75-84'
    
    if '85+' in col:
        return '85+'
    
g = df_all.groupby(combine_ages,axis=1)
print (g.groups)
###

### PATTERN ###
### div. grp deaths by grp mort and mult by nr of weeks in year ###

def calc_pop(grp):
    return (grp[grp.columns[0]] / grp[grp.columns[1]] * 52)
    
age_pop = g.apply(calc_pop)
###

age_pop = age_pop.dropna().astype(int)
age_pop

In [ ]:
age_grp_pop = age_pop.groupby(['Country','Year']).mean()
age_grp_pop


In [ ]:
age_dead_grps = df_all.groupby(combine_ages,axis=1)

def calc_dead(grp):
    return (grp[grp.columns[0]].groupby(['Country','Year']).sum())

age_grp_dead = age_dead_grps.apply(calc_dead)
age_grp_dead

In [ ]:
age_grp_mort = age_grp_dead / age_grp_pop
age_grp_mort

In [ ]:
std_pop = age_grp_pop.loc['SWE'].loc[2019]
std_pop

In [ ]:
std_age_grp_dead = std_pop * age_grp_mort
std_age_grp_dead

In [ ]:
std_dead = std_age_grp_dead.sum(axis=1)
std_dead

In [ ]:
# drop all NAN years #

std_dead = std_dead.loc[std_dead.index.get_level_values(1) >= 2007]
std_dead

In [ ]:
std_dead = std_dead.unstack().T

In [ ]:
std_dead.drop(2021,axis=0,inplace=True)
std_dead

In [ ]:
std_dead.plot(figsize=(18,12),style='o--')

In [ ]:
age_adj_mort = std_dead / std_pop.sum()
age_adj_mort

In [ ]:
title = 'Nordic Countries Age Adjusted Mortality [std.pop: SWE 2019] 2007-2020\nDataSource :mortality.org'
age_adj_mort.plot(figsize=(18,12),style='o--',title=title)
plt.ylabel('age adj. mortality')
plt.savefig('mortality_org_nordics_age_adj_mort_2007_2020.jpg',format='jpg')

In [ ]:
age_adj_mort_baseline = age_adj_mort.loc[2015:2018].mean()
age_adj_mort_baseline

In [ ]:
age_adj_mort_growth = age_adj_mort / age_adj_mort_baseline
age_adj_mort_growth

In [ ]:
age_adj_mort_growth.plot(figsize=(18,12),style='o--')

In [ ]:
x = 10000 * 200000
print ('{:.2E}'.format(x))

In [ ]:
age_grp_dead = age_grp_dead.loc[(age_grp_dead.index.get_level_values(1) >= 2007) &\
                                (age_grp_dead.index.get_level_values(1) <= 2020)]
age_grp_dead.loc['SWE']

In [ ]:
age_grp_dead

In [ ]:
### PATTERN ###
avg_dead_pairwise_years = age_grp_dead.groupby('Country').apply(lambda x : (x + x.shift())/2).dropna()
avg_dead_pairwise_years

In [ ]:
pairwise_age_grp_mort = (avg_dead_pairwise_years / age_grp_pop).dropna()
pairwise_age_grp_mort 

In [ ]:
pairwise_std_dead = pairwise_age_grp_mort * std_pop
pairwise_std_mort = pairwise_std_dead.sum(axis=1) / std_pop.sum()
pairwise_std_mort = pairwise_std_mort.unstack().T
pairwise_std_mort

In [ ]:
### 07/08...19/20 ###
xticks = ['07/08','08/09','09/10','10/11','11/12','12/13',
         '13/14','14/15','15/16','16/17','17/18','18/19','19/20']

title = 'Nordic Countries Age Adj. Mortality [std.pop SWE 2019] pairwise years\nDataSource : mortality.org'
ax = pairwise_std_mort.plot(figsize=(18,12),style='o--',title=title)
plt.ylabel('age adj. mortality')
plt.xlabel('year pair')

_= ax.set_xticks(pairwise_std_mort.index)
_= ax.set_xticklabels(xticks)